In [1]:
!pip install numpy opencv-python ultralytics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 723.1/723.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 862.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 924.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [2]:
import sys
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO

In [3]:
video_path = '/content/istockphoto-680128580-640_adpp_is.mp4'

In [13]:
# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

# Open the video file
cap = cv2.VideoCapture(video_path)

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video_path = video_path.split('.')[0] + '_tracked.mp4'
frame_rate = cap.get(cv2.CAP_PROP_FPS)
frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
out = cv2.VideoWriter(output_video_path, fourcc, frame_rate, frame_size)

In [14]:
track_history = defaultdict(lambda: [])
while cap.isOpened():
    success, frame = cap.read()
    if success:
        results = model.track(frame, persist=True)

        if results[0].boxes and results[0].boxes.id is not None:
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()

            annotated_frame = results[0].plot()

            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))
                if len(track) > 30:
                    track.pop(0)

                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

            out.write(annotated_frame)
    else:
        break

cap.release()
out.release()
cv2.destroyAllWindows()


0: 384x640 2 cars, 395.7ms
Speed: 3.4ms preprocess, 395.7ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 274.9ms
Speed: 5.6ms preprocess, 274.9ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 241.3ms
Speed: 5.4ms preprocess, 241.3ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 169.9ms
Speed: 4.6ms preprocess, 169.9ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 183.9ms
Speed: 6.4ms preprocess, 183.9ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 179.8ms
Speed: 8.5ms preprocess, 179.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 176.0ms
Speed: 5.1ms preprocess, 176.0ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 177.3ms
Speed: 4.0ms preprocess, 177.3ms inference, 1.5ms postprocess per image at shape (1, 3, 38

In [12]:
track_history = defaultdict(lambda: [])
while cap.isOpened():
    success, frame = cap.read()
    if success:
        results = model.track(frame, persist=True)

        if results[0].boxes and results[0].boxes.id is not None:
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()

            annotated_frame = results[0].plot()

            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))
                if len(track) > 30:
                    track.pop(0)

                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

            out.write(annotated_frame)  # Write the annotated frame to the output video
    else:
        break

cap.release()
out.release()
cv2.destroyAllWindows()


